In [1]:
import os
import csv
import networkx as nx
import pandas as pd
from random import shuffle

from IPython.display import display, SVG

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdDepictor
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem import Draw

In [14]:
data_3CLPro_7BQY_A_1_F = pd.read_csv("docking_data_idr/3CLPro_7BQY_A_1_F.csv")
data_NSP15_6W01_AB_1_F = pd.read_csv("docking_data_idr/NSP15_6W01_AB_1_F.csv")
data_NSP16_6W61_A_1_H = pd.read_csv("docking_data_idr/NSP16_6W61_A_1_H.csv")
data_PLPro_6W9C_A_2_F = pd.read_csv("docking_data_idr/PLPro_6W9C_A_2_F.csv")
smiles = pd.read_csv("docking_data_idr/ORD.mini.sorted.csv", header = 0, names = ["SMILE", "Title"])
print(data_3CLPro_7BQY_A_1_F.shape, data_NSP15_6W01_AB_1_F.shape, data_NSP16_6W61_A_1_H.shape,\
      data_PLPro_6W9C_A_2_F.shape, smiles.shape)

(2907, 13) (2928, 13) (2921, 13) (2869, 13) (2935, 2)


In [15]:
# There are some duplicates.
data_3CLPro_7BQY_A_1_F[data_3CLPro_7BQY_A_1_F[["Title"]].duplicated()]

,Title,Score_vina,rank_vina,percentile_vina,Score_fred,rank_fred,percentile_fred,Score_dock,rank_dock,percentile_dock,local_idr,global_idr,max_percentile
2491,ORD-MCULE-3967726014_1,-7.25,422.0,0.144,-10.961745,2595.0,0.884,-35.589,1505.0,0.483,0.934231,0.54347,0.884


In [16]:
smiles_3CLPro_7BQY_A_1_F = pd.merge(data_3CLPro_7BQY_A_1_F, smiles, how = 'inner', on = "Title")
smiles_NSP15_6W01_AB_1_F = pd.merge(data_NSP15_6W01_AB_1_F, smiles, how = 'inner', on = "Title")
smiles_NSP16_6W61_A_1_H  = pd.merge(data_NSP16_6W61_A_1_H, smiles, how = 'inner', on = "Title")
smiles_PLPro_6W9C_A_2_F  = pd.merge(data_PLPro_6W9C_A_2_F, smiles, how = 'inner', on = "Title")
print(smiles_3CLPro_7BQY_A_1_F.shape, smiles_NSP15_6W01_AB_1_F.shape, smiles_NSP16_6W61_A_1_H.shape,\
      smiles_PLPro_6W9C_A_2_F.shape)

(2908, 14) (2929, 14) (2922, 14) (2870, 14)


In [17]:
smiles_3CLPro_7BQY_A_1_F = smiles_3CLPro_7BQY_A_1_F.rename(columns={'Score_vina':'vina_3CLPro', 'Score_fred':'fred_3CLPro','Score_dock':'dock_3CLPro'})
smiles_NSP15_6W01_AB_1_F = smiles_NSP15_6W01_AB_1_F.rename(columns={'Score_vina':'vina_NSP15', 'Score_fred':'fred_NSP15','Score_dock':'dock_NSP15'})
smiles_NSP16_6W61_A_1_H  = smiles_NSP16_6W61_A_1_H .rename(columns={'Score_vina':'vina_NSP16', 'Score_fred':'fred_NSP16','Score_dock':'dock_NSP16'})
smiles_PLPro_6W9C_A_2_F  = smiles_PLPro_6W9C_A_2_F .rename(columns={'Score_vina':'vina_PLPro', 'Score_fred':'fred_PLPro','Score_dock':'dock_PLPro'})
print(smiles_3CLPro_7BQY_A_1_F.shape, smiles_NSP15_6W01_AB_1_F.shape, smiles_NSP16_6W61_A_1_H.shape,\
      smiles_PLPro_6W9C_A_2_F.shape)

(2908, 14) (2929, 14) (2922, 14) (2870, 14)


In [22]:
from functools import reduce
_ = reduce(lambda left,right: pd.merge(left,right,on="Title", how = 'inner'), \
       [smiles_3CLPro_7BQY_A_1_F.iloc[:,[0,-1,1,4,7]], smiles_NSP15_6W01_AB_1_F.iloc[:,[0,-1,1,4,7]],\
        smiles_NSP16_6W61_A_1_H.iloc[:,[0,-1,1,4,7]], smiles_PLPro_6W9C_A_2_F.iloc[:,[0,-1,1,4,7]]])
_ = _[~_[['Title']].duplicated()] 

In [33]:
_

,Title,SMILE_x,vina_3CLPro,fred_3CLPro,dock_3CLPro,SMILE_y,vina_NSP15,fred_NSP15,dock_NSP15,SMILE_x,vina_NSP16,fred_NSP16,dock_NSP16,SMILE_y,vina_PLPro,fred_PLPro,dock_PLPro
0,ORD-MCULE-2409941333_1,c1ccc(cc1)C[C@@H](C(=O)NCCC2=CCCCC2)NC(=O)[C@@...,-12.92,-10.490352,-28.523,c1ccc(cc1)C[C@@H](C(=O)NCCC2=CCCCC2)NC(=O)[C@@...,-9.72,-4.357690,-10.822,c1ccc(cc1)C[C@@H](C(=O)NCCC2=CCCCC2)NC(=O)[C@@...,-11.51,-7.172558,12.547,c1ccc(cc1)C[C@@H](C(=O)NCCC2=CCCCC2)NC(=O)[C@@...,-10.28,-8.093352,71.367
1,ORD-MCULE-6426577239_1,c1ccc(cc1)C[C@@H](c2[nH]c3ccccc3n2)NC(=O)CSc4c...,-12.63,-11.711597,-41.716,c1ccc(cc1)C[C@@H](c2[nH]c3ccccc3n2)NC(=O)CSc4c...,-8.88,-6.441950,-37.089,c1ccc(cc1)C[C@@H](c2[nH]c3ccccc3n2)NC(=O)CSc4c...,-10.13,-10.273417,-43.642,c1ccc(cc1)C[C@@H](c2[nH]c3ccccc3n2)NC(=O)CSc4c...,-10.79,-8.173996,-21.824
2,ORD-MCULE-9438397769_1,CC(C)C[C@@H](C(=O)NCCc1ccccc1F)NC(=O)[C@@H]2Cc...,-12.45,-11.247527,-33.751,CC(C)C[C@@H](C(=O)NCCc1ccccc1F)NC(=O)[C@@H]2Cc...,-7.99,-4.399041,-14.916,CC(C)C[C@@H](C(=O)NCCc1ccccc1F)NC(=O)[C@@H]2Cc...,-9.62,-8.082316,-41.175,CC(C)C[C@@H](C(=O)NCCc1ccccc1F)NC(=O)[C@@H]2Cc...,-10.70,-7.750196,-23.119
3,ORD-MCULE-5858157644_1,c1ccc(cc1)C[C@@H](c2[nH]c3ccccc3n2)NC(=O)c4ccc...,-12.25,-11.432585,-54.831,c1ccc(cc1)C[C@@H](c2[nH]c3ccccc3n2)NC(=O)c4ccc...,-7.56,-7.147294,-39.494,c1ccc(cc1)C[C@@H](c2[nH]c3ccccc3n2)NC(=O)c4ccc...,-8.61,-7.385507,-2.789,c1ccc(cc1)C[C@@H](c2[nH]c3ccccc3n2)NC(=O)c4ccc...,-10.01,-6.738115,33.312
4,ORD-MCULE-3098689138_1,C[C@@H](C(=O)NCc1c(cccc1Cl)F)NC(=O)[C@@H]2Cc3c...,-12.24,-9.409821,-40.988,C[C@@H](C(=O)NCc1c(cccc1Cl)F)NC(=O)[C@@H]2Cc3c...,-8.33,-5.266155,-33.691,C[C@@H](C(=O)NCc1c(cccc1Cl)F)NC(=O)[C@@H]2Cc3c...,-9.57,-9.302455,-13.953,C[C@@H](C(=O)NCc1c(cccc1Cl)F)NC(=O)[C@@H]2Cc3c...,-9.46,-8.937026,-29.394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3083,ORD-MCULE-5643868593_1,c1c(c(c(c(n1)C(=O)[O-])Cl)C#N)C(F)(F)F,-4.60,-6.305102,-31.447,c1c(c(c(c(n1)C(=O)[O-])Cl)C#N)C(F)(F)F,-5.11,-5.040735,-20.102,c1c(c(c(c(n1)C(=O)[O-])Cl)C#N)C(F)(F)F,-4.51,-8.041244,-24.790,c1c(c(c(c(n1)C(=O)[O-])Cl)C#N)C(F)(F)F,-5.98,-8.205172,-25.764
3084,ORD-MCULE-1089692040_1,c1[nH]c2c(nc(nc2n1)N)C(=O)N,-4.53,-6.263764,-23.774,c1[nH]c2c(nc(nc2n1)N)C(=O)N,-6.07,-6.131366,21.338,c1[nH]c2c(nc(nc2n1)N)C(=O)N,-6.32,-7.472608,-23.224,c1[nH]c2c(nc(nc2n1)N)C(=O)N,-6.21,-7.573913,-26.654
3085,ORD-MCULE-6430306176_1,Cn1c2ccccc2c(c1O)C(C(=O)[O-])C(=O)[O-],-4.49,-6.189229,-32.065,Cn1c2ccccc2c(c1O)C(C(=O)[O-])C(=O)[O-],-5.45,-5.956166,-26.022,Cn1c2ccccc2c(c1O)C(C(=O)[O-])C(=O)[O-],-5.42,-7.365056,-33.115,Cn1c2ccccc2c(c1O)C(C(=O)[O-])C(=O)[O-],-5.66,-8.748167,-34.165
3086,ORD-MCULE-2100796872_1,C(CSC(=[NH2+])N)CSC(=[NH2+])N,-4.32,-10.003424,-25.518,C(CSC(=[NH2+])N)CSC(=[NH2+])N,-4.43,-8.813179,-16.988,C(CSC(=[NH2+])N)CSC(=[NH2+])N,-4.68,-10.444361,-22.261,C(CSC(=[NH2+])N)CSC(=[NH2+])N,-4.47,-8.663182,-21.852


In [35]:
_.iloc[:,[0,1,2,3,4,6,7,8,10,11,12]].to_csv("Multilabel_data.csv", index = False)

In [42]:
#Thresholding
smiles_3CLPro_7BQY_A_1_F = smiles_3CLPro_7BQY_A_1_F[smiles_3CLPro_7BQY_A_1_F['global_idr'] < 0.05].sort_values('global_idr')
smiles_NSP15_6W01_AB_1_F = smiles_NSP15_6W01_AB_1_F[smiles_NSP15_6W01_AB_1_F['global_idr'] < 0.05].sort_values('global_idr')
smiles_NSP16_6W61_A_1_H  = smiles_NSP16_6W61_A_1_H[smiles_NSP16_6W61_A_1_H['global_idr'] < 0.05].sort_values('global_idr')
smiles_PLPro_6W9C_A_2_F  = smiles_PLPro_6W9C_A_2_F[smiles_PLPro_6W9C_A_2_F['global_idr'] < 0.05].sort_values('global_idr')

In [44]:
mols_3CLPro_7BQY_A_1_F = [Chem.MolFromSmiles(i) for i in smiles_3CLPro_7BQY_A_1_F['SMILE']]
mols_NSP15_6W01_AB_1_F = [Chem.MolFromSmiles(i) for i in smiles_NSP15_6W01_AB_1_F['SMILE']]
mols_NSP16_6W61_A_1_H  = [Chem.MolFromSmiles(i) for i in smiles_NSP16_6W61_A_1_H['SMILE']]
mols_PLPro_6W9C_A_2_F  = [Chem.MolFromSmiles(i) for i in smiles_PLPro_6W9C_A_2_F['SMILE']]

In [127]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

with PdfPages('IDR_Threshold_3CLPro_7BQY_A_1_F.pdf') as pdf:
    for i in range(len(mols_3CLPro_7BQY_A_1_F)):
        #plt.rc('text', usetex=False)
        fig = Draw.MolToMPL(mols_3CLPro_7BQY_A_1_F[i], size=(300,300))
        plt.title(str(i+1))
        plt.text(0.01,0.1,smiles_3CLPro_7BQY_A_1_F['SMILE'][j])
        plt.text(0.01,0.05,smiles_3CLPro_7BQY_A_1_F['Title'][j])
        pdf.savefig(fig, bbox_inches = "tight")  # saves the current figure into a pdf page
        plt.close()
        
with PdfPages('IDR_Threshold_NSP15_6W01_AB_1_F.pdf') as pdf:
    for i in range(len(mols_NSP15_6W01_AB_1_F)):
        #plt.rc('text', usetex=False)
        fig = Draw.MolToMPL(mols_NSP15_6W01_AB_1_F[i], size=(300,300))
        plt.title(str(i+1))
        plt.text(0.01,0.1,smiles_NSP15_6W01_AB_1_F['SMILE'][j])
        plt.text(0.01,0.05,smiles_NSP15_6W01_AB_1_F['Title'][j])
        pdf.savefig(fig, bbox_inches = "tight")  # saves the current figure into a pdf page
        plt.close()
        
with PdfPages('IDR_Threshold_NSP16_6W61_A_1_H.pdf') as pdf:
    for i in range(len(mols_NSP16_6W61_A_1_H)):
        #plt.rc('text', usetex=False)
        fig = Draw.MolToMPL(mols_NSP16_6W61_A_1_H[i], size=(300,300))
        plt.title(str(i+1))
        plt.text(0.01,0.1,smiles_NSP16_6W61_A_1_H['SMILE'][j])
        plt.text(0.01,0.05,smiles_NSP16_6W61_A_1_H['Title'][j])
        pdf.savefig(fig, bbox_inches = "tight")  # saves the current figure into a pdf page
        plt.close()
        
with PdfPages('IDR_Threshold_PLPro_6W9C_A_2_F.pdf') as pdf:
    for i in range(len(mols_PLPro_6W9C_A_2_F)):
        #plt.rc('text', usetex=False)
        fig = Draw.MolToMPL(mols_PLPro_6W9C_A_2_F[i], size=(300,300))
        plt.title(str(i+1))
        plt.text(0.01,0.1,smiles_PLPro_6W9C_A_2_F['SMILE'][j])
        plt.text(0.01,0.05,smiles_PLPro_6W9C_A_2_F['Title'][j])
        pdf.savefig(fig, bbox_inches = "tight")  # saves the current figure into a pdf page
        plt.close()

In [70]:
# Multiple targets
multiple_targets = pd.read_csv("docking_data_idr/multiple_targets.csv")

In [100]:
multiple_targets

,Interaction,Molecules
0,"('NSP15_6W01_AB_1_F', '3CLPro_7BQY_A_1_F')","{'ORD-MCULE-6477680903_1', 'ORD-MCULE-37738418..."
1,"('NSP15_6W01_AB_1_F', 'NSP16_6W61_A_1_H')","{'ORD-MCULE-6477680903_1', 'ORD-MCULE-42620917..."
2,"('NSP15_6W01_AB_1_F', 'PLPro_6W9C_A_2_F')","{'ORD-MCULE-7416131701_1', 'ORD-MCULE-93119370..."
3,"('3CLPro_7BQY_A_1_F', 'NSP16_6W61_A_1_H')","{'ORD-MCULE-1562586832_1', 'ORD-MCULE-64776809..."
4,"('3CLPro_7BQY_A_1_F', 'PLPro_6W9C_A_2_F')","{'ORD-MCULE-5739750381_1', 'ORD-MCULE-91301999..."
5,"('NSP16_6W61_A_1_H', 'PLPro_6W9C_A_2_F')","{'ORD-MCULE-7968599617_1', 'ORD-MCULE-74161317..."
6,"('NSP15_6W01_AB_1_F', '3CLPro_7BQY_A_1_F', 'NS...","{'ORD-MCULE-6977725985_1', 'ORD-MCULE-74161317..."
7,"('NSP15_6W01_AB_1_F', '3CLPro_7BQY_A_1_F', 'PL...","{'ORD-MCULE-7416131701_1', 'ORD-MCULE-93119370..."
8,"('NSP15_6W01_AB_1_F', 'NSP16_6W61_A_1_H', 'PLP...","{'ORD-MCULE-7416131701_1', 'ORD-MCULE-93119370..."
9,"('3CLPro_7BQY_A_1_F', 'NSP16_6W61_A_1_H', 'PLP...","{'ORD-MCULE-7416131701_1', 'ORD-MCULE-93119370..."


In [125]:
with PdfPages('Multiple_Targets.pdf') as pdf:
    for i in range(multiple_targets.shape[0]):
        interaction = multiple_targets['Interaction'][i]
        mol_data = multiple_targets['Molecules'][i]
        
        #Filtered Dataframe according to multipletargets data
        arr = mol_data.replace("'","").replace("{","").replace("}","").replace(" ","").split(",")
        _ = pd.DataFrame({'Title':arr})
        filtered_df = pd.merge(smiles, _, how = 'inner', on = "Title")
        smile_filter = filtered_df['SMILE'].values
        title_filter = filtered_df['Title'].values

        mols = [Chem.MolFromSmiles(i) for i in smile_filter]
        for j in range(len(mols)):
            fig = Draw.MolToMPL(mols[j], size = (300,300))
            plt.title(interaction)
            plt.text(0.01,0.1,smile_filter[j])
            plt.text(0.01,0.05,title_filter[j])
            pdf.savefig(fig, bbox_inches = "tight")
            plt.close()